*Link to presentation*: https://www.youtube.com/watch?v=nJ3H-DwWlls

In [ ]:
import random
import copy
import unittest

# **Bulls and Cows Game (using Genetic Algorithm)**


*   In the Bulls and Cows Game, Player 1 provides a sequence of digits from 0 to 9, which is the solution. Player 2 then writes down a guess. Player 1 will assign a score to the guess based on the number of "Bulls" and "Cows" in the guess.
*   A "Bull" is a digit that is the correct value and in the correct position.
*   A "Cow" is a digit that is the correct value but in the wrong position.
*   Based on the score given by Player 1, Player 2 will either win the game since they guessed the solution, or they will keep guessing and getting a score back from Player 2 until they guess the correct number.
*   To see our scoring system, look at the documentation for "Chromosome Class"





# **Chromosome Class**

In [ ]:
class Chromosome:
  '''
  Represents an individual in the population. Each individual is a sequence of 6 digits from 0 to 9 and has a fitness score based
  on the game, "Bulls and Cows":
    - if the value and position of a digit is correct, that is +5 to the fitness score
    - else if the value of a digit is correct but the position is wrong, that is +1 to the fitness score
    - an optimal individual has a fitness score of 30 (6 digits with correct value and position)
  '''
    def __init__(self, genes, fitness):
        self.Genes = genes # gene is a sequence of 6 digits from 0 to 9
        self.Fitness = fitness # fitness score based on the game, "Bulls and Cows"

# **_generate_parent**
Randomly generates 10 individuals for the first generation of the population.


*   **length** int: number of genes in each chromosome
*   **chromosome_set** str: set of possible values for each gene ("0123456789")
*   **get_fitness** method: returns fitness score of a given individual

**returns**

*   **parents** List[Chromosome]: first generation of the population





In [ ]:
def _generate_parent(length, chromosome_set, get_fitness):
    parents = []

    # initialize all parents (population size of 10)
    for i in range(0, 10):
        genes = []
        
        # assign random values for the genes
        while len(genes) < length:
            sample_size = min(length - len(genes), len(chromosome_set))
            genes.extend(random.sample(chromosome_set, sample_size))

        # create new parent based on the random genes and their fitness
        parents.append(Chromosome(genes, get_fitness(genes)))

    return parents

# **_mutate**
Mutates a random gene in the given individual.


*   **parent** Chromosome: individual who's chromosome is being mutated
*   **gene_set** str: set of possible values for each gene ("0123456789")
*   **get_fitness** method: returns fitness score of a given individual

**returns**

*   **mutated_individual** Chromosome: given individual except with a gene mutated

In [ ]:
def _mutate(parent, gene_set, get_fitness):
    # get the to-be offspring's genes
    offspring_genes = parent.Genes[:]

    # randomly mutate (change) a value within the gene
    index = random.randrange(0, len(parent.Genes))
    new_gene, alternate = random.sample(gene_set, 2)
    offspring_genes[index] = alternate if new_gene == offspring_genes[index] else new_gene
    
    # update fitness after mutation
    fitness = get_fitness(offspring_genes)
    
    return Chromosome(offspring_genes, fitness)

# **_generate_offspring**

Create offspring from given generation of parents. Only choose parents with the highest fitness scores. Fixed cross-over rate of 0.2 (20%) and mutation rate of 0.15 (15%).

*   **parent_list** List[Chromosome]: parents that offspring are generated from
*   **gene_set** str: set of possible values for each gene ("0123456789")
*   **get_fitness** method: returns fitness score of a given individual

**returns**

*   **offspring_list** list[Chromosome]: offspring of parents

In [ ]:
def _generate_offspring(parent_list, gene_set, get_fitness):
    offspring_list = []
    fitness_percent_list = []
    fitness_accum_list = []
    fitness_sum = 0
    
    # evaluate fitness of each parent
    for parent in parent_list:
        fitness_sum += parent.Fitness
    for parent in parent_list:
        fitness_percent_list.append(parent.Fitness / fitness_sum)

    # get all fitness scores (as a percentage of the total fitness)
    fitness_sum = 0
    for fitness_percent in fitness_percent_list:
        fitness_sum += fitness_percent
        fitness_accum_list.append(fitness_sum)

    # select individuals for reproduction based on fitness
    # iterate through all current inviduals in the population
    for i in range(0, 10):
        rand = random.random()
        before = 0
        
        # reproduce (add offspring) based on previous and current fitness sum
        for j in range(0, len(fitness_accum_list)):
            if rand > before and rand <= fitness_accum_list[j]:
                offspring_list.append(copy.deepcopy(parent_list[j]))
                break
            before = fitness_accum_list[j]

    # cross over parents' info
    crossover_rate = 0.20
    selected = None
    for i in range(0, len(offspring_list)):
        rand = random.random()
        if rand < crossover_rate:
            if selected is None:
                selected = i
            else:
                offspring_list[selected].Genes[2:], offspring_list[i].Genes[2:] = \
                    offspring_list[i].Genes[2:], offspring_list[selected].Genes[2:]
                selected = None

        offspring_list[i].Fitness = get_fitness(offspring_list[i].Genes)

    # mutate offspring's info
    mutate_rate = 0.15
    for i in range(0, len(offspring_list)):
        rand = random.random()
        if rand < mutate_rate:
            offspring = _mutate(offspring_list[i], gene_set, get_fitness)
            del offspring_list[i]
            offspring_list.append(offspring)
   
    return offspring_list


# **get_best**
Runs Genetic Algorithm to play a game of "Bulls and Cows." Only terminates once the average fitness score reaches optimal fitness aka all offspring's chromosomes are the correct sequence (average fitness score = 30).


*   **get_fitness** method: returns fitness score of a given individual
*   **goal_len** int: number of genes in each chromosome
*   **optimal_fitness** int: average fitness score (of the population) that the genetic algorithm terminates at
*   **gene_set** str: set of possible values for each gene ("0123456789")
*   **display** method: rints out a given individual's chromosome, the number of bulls and cows in the chromosome ("{bulls}/{cows}"), and the individual's fitness score

**returns**
*   **final_generation** List[Chromosome]: last generation of the Genetic Algorithmm (after termination)



In [ ]:
def get_best(get_fitness, goal_len, optimal_fitness, gene_set, display):
    random.seed()

    # initialize individuals of the population (to be overwritten with each generation)
    best_parent_list = _generate_parent(goal_len, gene_set, get_fitness)

    print("\nINITIAL POPULATION:\n")
    display(best_parent_list)
    print(f"\n\nThe optimal fitness is {optimal_fitness}. As you can see, not very optimal... yet!")
    print("Let's use our genetic algorithm and see what happens...\n\n")
    
    # prepare for the algorithm
    print("{:9s} {}".format("Gen #", "Avg fitness"))
    print("---------------------")
    gen_count = 0
    maximum_average = 0
    offspring_list = None

    # iterate through the loop until convergence critera are met
    while True:
        # increment the generation count with each iteration
        gen_count += 1

        # generate offspring from parents (through crossover and mutuation)
        offspring_list = _generate_offspring(best_parent_list, gene_set, get_fitness)

        # evaluate the fitness of offspring
        fitness_sum = 0
        for offspring in offspring_list:
            fitness_sum += offspring.Fitness
        average = fitness_sum / 10
        
        # print generation number and current average fitness
        print("{:<9d} {}".format(gen_count, average))

        # select individuals for the next generation and replace old generation with new population
        if average > maximum_average:
            best_parent_list = offspring_list
            maximum_average = average

        # once the average fitness of the population reaches optimal fitness (convergence criterion), terminate the loop
        if average >= optimal_fitness:
            break
    
    print("---------------------\n")
    return offspring_list

# **get_fitness**
Determines the fitness score of the given individual:
*   if the value and position of a digit is correct, that is +5 to the fitness score
*   else if the value of a digit is correct but the position is wrong, that is +1 to the fitness score
*   an optimal individual has a fitness score of 30 (6 digits with correct value and position)



*   **guess** List[str]: the individual's chromosome sequence
*   **goal** List[str]: the optimal chromosome sequence (solution)

**returns**


*   **fitness** int: fitness score of the given individual



In [ ]:
def get_fitness(guess, goal):
    # determine fitness based on expected vs. actual
    fitness = 0
    for expected, estimate in zip(goal, guess):
        if expected == estimate:
            fitness += 5
        elif estimate in goal:
            fitness += 1

    return fitness

# **pick_num**
Randomly generates a number with "length" digits. Will repeat digits if the number has more than 10 digits or "is_duplicate_allowed" is true.


*   **length** int: number of digits that the number will have
*   **is_duplicate_allowed** list: if true, number generated can repeat digits; if false, duplicates will be avoided unless "length" > 10

**returns**

*   **solution_sequence** str: randomly-generated number with "length" digits



In [ ]:
def pick_num(length, is_duplicate_allowed):
    if  length > 10 or is_duplicate_allowed is True:
        return ''.join(random.choice("0123456789")
                       for _ in range(length))

    bullscows_list = []
    num = random.randrange(0, 10)

    for i in range(length):
        while str(num) in bullscows_list:
            num = random.randrange(0, 10)
        bullscows_list.append(str(num))

    return ''.join(bullscows_list)

# **display**
Prints out given individual's chromosome, the number of bulls and cows in the chromosome ("{bulls}/{cows}"), and the fitness score of the individual.


*   **possible** Chromosome: individual in the population
*   **goal** str: sequence of digits that is the optimal sequence (solution)

**returns**

*   *None*: but prints out a nice visualization, as described in the description

In [ ]:
def display(possible, goal):
    bulls = 0
    cows = 0
    
    for expected, estimate in zip(goal, possible.Genes):
        if expected == estimate:
            bulls += 1
        elif estimate in goal:
            cows += 1

    if bulls == 0 and cows == 0:
        result = "out"
    else:
        result = "{}/{}".format(bulls, cows)

    print("{}\t{}\t{}".format(
        ''.join(possible.Genes),
        result,
        possible.Fitness,
        ))

# **display_list**
Prints out information about individuals in given generation and average fitness of given generation.


*   **possible_list** List[Chromosome]: generation of individuals 
*   **goal** str: sequence of digits that is the optimal sequence (solution)

**returns**

*   *None*: but prints out a nice visualization, as described in the description

In [ ]:
def display_list(possible_list, goal):
    fitness_sum = 0

    for possible in possible_list:
        display(possible, goal)
        fitness_sum += possible.Fitness

    print("-\nAverage fitness: {}".format(fitness_sum / len(possible_list)))

# **BullsCowsGames Class**

In [ ]:
class BullsCowsGames(unittest.TestCase):
    '''
    Represents a game of Bulls and Cows.  Establishes allowed digits for the sequence, determines solution sequence, runs a game, and
    formats visualization nicely for the user.
    '''
    gene_set = "0123456789"

    def test_bullscows(self):
        length = 6

        # pick a random number with length of `length` as a goal
        goal = pick_num(length, False)
        print("\n------------")
        print("GOAL: {}".format(goal))
        print("------------")
        self.guess_bullscows(goal)

    def guess_bullscows(self, goal):

        def fn_get_fitness(genes):
            return get_fitness(genes, goal)

        def fn_display(possible_list):
            display_list(possible_list, goal)

        # get best offspring
        optimal_fitness = len(goal) * 5
        offspring_list = get_best(fn_get_fitness, len(goal), optimal_fitness,
                                self.gene_set, fn_display)

        print("FINAL POPULATION:\n")
        fn_display(offspring_list)

**Main Function**

In [ ]:
def main():

    def fn_get_fitness(genes):
        return get_fitness(genes, goal)

    length = len(goal)
    optimal_fitness = length * 5
    
    best = get_best(fn_get_fitness, length, optimal_fitness, gene_set, fn_display)
    display_list(best, goal)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.135s

OK



------------
GOAL: 281743
------------

INITIAL POPULATION:

481675	2/2	12
462789	1/3	8
869135	0/3	3
290145	2/1	11
105438	0/4	4
068453	1/2	7
187904	1/3	8
310684	0/4	4
586230	1/2	7
305726	1/2	7
-
Average fitness: 7.1


The optimal fitness is 30. As you can see, not very optimal... yet!
Let's use our genetic algorithm and see what happens...


Gen #     Avg fitness
---------------------
1         7.6
2         8.0
3         12.3
4         13.1
5         13.5
6         13.7
7         14.6
8         14.2
9         15.6
10        16.7
11        16.8
12        16.2
13        16.5
14        15.9
15        16.8
16        15.2
17        16.8
18        16.6
19        15.8
20        16.3
21        17.2
22        16.5
23        17.6
24        16.4
25        16.8
26        16.4
27        17.5
28        17.4
29        17.6
30        17.1
31        17.4
32        18.1
33        18.3
34        17.9
35        17.4
36        17.7
37        18.6
38        17.8
39        18.4
40        18.5
41        19.